<a href="https://colab.research.google.com/github/brendanlooker/colab-examples/blob/main/bq/BQ_vector_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Create BQ model

create or replace model `CustomerReview.Embeddings`
remote with connection `projects/qwiklabs-gcp-00-75cf23400ea8/locations/us/connections/embedding_conn`
options (ENDPOINT = 'text-embedding-005');

In [ ]:
# Create a table and load with embeddings generated using the model and the text provided

create or replace table `qwiklabs-gcp-00-75cf23400ea8.CustomerReview.customer_reviews_embedded` as
select * from ML.GENERATE_EMBEDDING(
  model `CustomerReview.Embeddings`,
  (select review_text as content from `qwiklabs-gcp-00-75cf23400ea8.CustomerReview.customer_reviews`)
)

In [ ]:
# Create an index of the vector search space

CREATE OR REPLACE VECTOR INDEX `CustomerReview.reviews_index`
ON `CustomerReview.customer_reviews_embedded`(ml_generate_embedding_result)
OPTIONS (distance_type = 'COSINE', index_type = 'IVF');

In [ ]:
# Search the vector space and retrieve the similar items

CREATE OR REPLACE TABLE `CustomerReview.vector_search_result` AS
SELECT
    query.query,
    base.content
FROM
    VECTOR_SEARCH(
        TABLE `CustomerReview.customer_reviews_embedded`,
        'ml_generate_embedding_result',
        (
            SELECT
                ml_generate_embedding_result,
                content AS query
            FROM
                ML.GENERATE_EMBEDDING(
                    MODEL `CustomerReview.Embeddings`,
                    (SELECT 'service' AS content)
                )
        ),
        top_k => 5,
        options => '{"fraction_lists_to_search": 0.01}'
    );

In [ ]:
# Generate an improved answer, create a Gemini Model

CREATE OR REPLACE MODEL `CustomerReview.Gemini`
REMOTE WITH CONNECTION `us.embedding_conn`
OPTIONS (ENDPOINT = 'gemini-2.0-flash');

In [ ]:
# To enhance Gemini's responses, provide it with relevant and recent data retrieved from the vector search by running the following query

SELECT
    ml_generate_text_llm_result AS generated
FROM
    ML.GENERATE_TEXT(
        MODEL `CustomerReview.Gemini`,
        (
            SELECT
                CONCAT(
                    'Summarize what customers think about our services',
                    STRING_AGG(FORMAT('review text: %s', base.content), ',\n')
                ) AS prompt
            FROM
                `CustomerReview.vector_search_result` AS base
        ),
        STRUCT(
            0.4 AS temperature,
            300 AS max_output_tokens,
            0.5 AS top_p,
            5 AS top_k,
            TRUE AS flatten_json_output
        )
    );